In [265]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import scipy.stats as stats

In [266]:
batting_2024 = pd.read_csv("/Users/benboda/Downloads/GitHub/Econometrics_Term_Paper/Yearly_Data/2024batting.csv")
pitching_2024 = pd.read_csv("/Users/benboda/Downloads/GitHub/Econometrics_Term_Paper/Yearly_Data/2024pitching.csv")
fielding_2024 = pd.read_csv("/Users/benboda/Downloads/GitHub/Econometrics_Term_Paper/Yearly_Data/2024fielding.csv")
win_totals = pd.read_csv("/Users/benboda/Downloads/GitHub/Econometrics_Term_Paper/Yearly_Data/win_totals.csv")

In [267]:
initials_dict = {"ARI": "Arizona Diamondbacks", "ATL": "Atlanta Braves", "BAL": "Baltimore Orioles", "BOS": "Boston Red Sox", "CHC": "Chicago Cubs", "CHW": "Chicago White Sox", "CIN": "Cincinnati Reds", "CLE": ["Cleveland Guardians", "Cleveland Indians"], "COL": "Colorado Rockies", "DET": "Detroit Tigers", "HOU": "Houston Astros", "KCR": "Kansas City Royals", "LAA": ["Los Angeles Angels", "Los Angeles Angels of Aneheim"], "LAD": "Los Angeles Dodgers", "MIA": "Miami Marlins", "MIL": "Milwaukee Brewers", "MIN": "Minnesota Twins", "NYM": "New York Mets", "NYY": "New York Yankees", "OAK": "Oakland Athletics", "PHI": "Philadelphia Phillies", "PIT": "Pittsburgh Pirates", "SDP": "San Diego Padres", "SFG": "San Francisco Giants", "SEA": "Seattle Mariners", "STL": "St. Louis Cardinals", "TBR": "Tampa Bay Rays", "TEX": "Texas Rangers", "TOR": "Toronto Blue Jays", "WSN": "Washington Nationals"}

In [268]:
win_totals = win_totals.melt(
    id_vars=["Year", "G"],
    var_name="Team", 
    value_name="Wins"
    )
win_totals.dropna(inplace=True)
win_totals.reset_index(drop=True, inplace=True)
win_totals.drop("G", axis=1, inplace=True)

In [269]:
win_totals2024 = win_totals[(win_totals["Year"] == 2024)].copy()
win_totals2024.reset_index(drop=True, inplace=True)

In [270]:
invert_dict = {team: abbr for abbr, teams in initials_dict.items() for team in (teams if isinstance(teams, list) else [teams])}
batting_2024["Team Abbreviation"] = batting_2024["Tm"].map(invert_dict)
batting_2024 = batting_2024.merge(win_totals2024, left_on="Team Abbreviation", right_on="Team", how="left", suffixes=("", "_wintotals"))
batting_2024.drop(["Team Abbreviation", "Year"], axis=1, inplace=True)
batting_2024.rename(columns={"Team": "Abbreviation", "Wins": "Wins"}, inplace=True)
batting_2024.insert(1, "Abbreviation", batting_2024.pop("Abbreviation"))
batting_2024.insert(2, "Wins", batting_2024.pop("Wins"))
batting_2024["Team"] = batting_2024["Tm"] + " " + "(" + batting_2024["Abbreviation"] + ")"
batting_2024.drop(["Tm", "Abbreviation"], axis=1, inplace=True)
batting_2024.insert(0, "Team", batting_2024.pop("Team"))
batting_2024.set_index("Team", inplace=True)
batting_2024.dropna(inplace=True)
batting_2024["XBH"] = batting_2024["2B"] + batting_2024["3B"] + batting_2024["HR"]
batting_2024.insert(12, "XBH", batting_2024.pop("XBH"))

In [271]:
X = batting_2024[["XBH", "SO", "SB", "LOB"]]
Y = batting_2024[["Wins"]]
model = sm.OLS(Y, sm.add_constant(X)).fit()
print(model.summary())
n = len(batting_2024)
k = X.shape[1]
df = n - k - 1
alpha = 0.05
critical_t = stats.t.ppf(1 - alpha/2, df)
print(f"Critical t-stat: {critical_t}")

                            OLS Regression Results                            
Dep. Variable:                   Wins   R-squared:                       0.631
Model:                            OLS   Adj. R-squared:                  0.572
Method:                 Least Squares   F-statistic:                     10.70
Date:                Thu, 28 Nov 2024   Prob (F-statistic):           3.42e-05
Time:                        22:10:59   Log-Likelihood:                -102.79
No. Observations:                  30   AIC:                             215.6
Df Residuals:                      25   BIC:                             222.6
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const        -56.4055     37.723     -1.495      0.1